In [1]:
# 벡터db에 연결
import numpy as np
import pandas as pd

from langchain.document_loaders import CSVLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
from langchain import FAISS
import faiss

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [34]:
qna_df = pd.read_csv('qa_data.csv')[['질문', '답변']]
qna_df['질문'] = qna_df['질문'].apply(lambda x: x.split('질문\n')[1]) # "질문\n" 제거
qna_df['답변'] = qna_df['답변'].apply(lambda x: x.split('답변\n')[1]) # "답변\n" 제거

In [35]:
qna_df

,질문,답변
0,"""지역화폐"" “지역화폐”의 의미?",“지역화폐”는 특정 지역 또는 특정 집단 내에서 자생적으로 형성된 지급수단으로서 「...
1,"""지역화폐"" 지역사랑상품권의 법화(법정통화) 여부?",『한국은행법』에 따르면 법화(法貨)는 한국은행이 발행한 한국은행권 및 주화만을 의미...
2,"""지역화폐"" 한국은행은 지역사랑상품권을 교환해주는지?",한국은행은 지역사랑상품권의 발행주체가 아니며 지역사랑상품권을 한국은행권 및 주화로 ...
3,"""지역화폐"" 지역사랑상품권의 발행주체 부도시 한국은행의 보상 여부?",지역사랑상품권은 한국은행에서 발행한 법화(法貨)가 아니므로 발행주체의 부도시 한국은...
4,발권출납 현용주화 발행연도,한국은행은 매년 화폐수급전망을 기초로 다음 연도에 필요한 화폐의 양을 한국조폐공사를...
...,...,...
90,발권출납 화폐도안의 광고물 이용행위 등에 대한 제한,화폐도안을 광고소재로 사용하는 것은 원칙적으로 바람직하지 않습니다. 왜냐하면 돈을 ...
91,지급결제 한국은행금융결제망(BOK-Wire+),한국은행금융결제망(BOK-Wire+)은 1994.12월부터 한국은행과 금융기관을 온...
92,경제통계 분기 경제성장률,전기비 성장률은 현재 분기의 경제상황이 직전 분기보다 나아지고 있는지 또는 나빠지고...
93,발권출납 화폐도안 선정기준,먼저 현용화폐의 경우 한국은행은 화폐 도안에 대한 국민여론과 각계 전문가들의 의견 ...


## FAISS에 따로 저장은 안하는 경우

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceTransformer 모델 로드
embedding_model = SentenceTransformer('jeonseonjin/embedding_BAAI-bge-m3')

# 쿼리 문장들에 대한 임베딩 벡터 생성
query_texts = qna_df['질문'].to_list()
document_embeddings = embedding_model.encode(query_texts)

In [77]:
# 쿼리 제시
new_query = '발권출납 옛날 돈과 현용화폐와의 교환'
new_query_embedding = embedding_model.encode([new_query])

# 코사인 유사도 계산
cos_sim = cosine_similarity(new_query_embedding, document_embeddings)

# 코사인 유사도 값이 가장 큰 질문 찾기
most_similar_idx = np.argmax(cos_sim)
similarity = np.round(cos_sim[0][most_similar_idx],2)

# 가장 비슷한 질문과 답변 가져오기
similar_query = query_texts[most_similar_idx]
similar_answer = qna_df[qna_df.index == most_similar_idx]['답변'].to_string(index=False)

print("가장 비슷한 질문 : ", similar_query)
print("가장 비슷한 질문의 거리 : ", similarity) #같은 질문에 대해서는 1.0이 나와야함
print("가장 비슷한 질문의 답: ", similar_answer)


가장 비슷한 질문 :  발권출납 옛날 돈과 현용화폐와의 교환
가장 비슷한 질문의 거리 :  1.0
가장 비슷한 질문의 답:  한국은행은 1962년 이후에 발행된 원 표시 화폐에 대해서만 액면 금액으로 교환해 드...


## FAISS에 저장

In [56]:
import faiss
import numpy as np

# 모든 벡터 정규화 (document_embeddings와 쿼리 모두)
document_embeddings = document_embeddings / np.linalg.norm(document_embeddings, axis=1, keepdims=True)

# FAISS 인덱스 생성 및 추가 (정규화된 벡터 사용)
embed_dim = document_embeddings.shape[1]
query_index = faiss.IndexFlatIP(embed_dim)
query_index.add(document_embeddings)
faiss.write_index(query_index, 'query_index.bin')

# 쿼리 제시
new_query = '발권출납 옛날 돈과 현용화폐와의 교환'
new_query_embedding = embedding_model.encode([new_query])

# 쿼리 벡터도 정규화
new_query_embedding = new_query_embedding / np.linalg.norm(new_query_embedding, axis=1, keepdims=True)

# FAISS 검색 (내적이 코사인 유사도와 동일)
distance, index = query_index.search(new_query_embedding, k=3)

# 가장 비슷한 질문의 인덱스와 유사도
most_similar_idx = index[0][0]
similarity = distance[0][0]  # 내적값은 코사인 유사도와 동일

# 가장 비슷한 질문과 답변 가져오기
similar_query = query_texts[most_similar_idx]
similar_answer = qna_df[qna_df.index == most_similar_idx]['답변'].to_string(index=False)

# 출력
print("가장 비슷한 질문 : ", similar_query)
print("가장 비슷한 질문의 유사도 (코사인 유사도): ", similarity)
print("가장 비슷한 질문의 답: ", similar_answer)


가장 비슷한 질문 :  발권출납 옛날 돈과 현용화폐와의 교환
가장 비슷한 질문의 유사도 (코사인 유사도):  1.0
가장 비슷한 질문의 답:  한국은행은 1962년 이후에 발행된 원 표시 화폐에 대해서만 액면 금액으로 교환해 드...


## 질의-답변을 함수로 제공

In [78]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# query-answer 함수 정의
def query_answer(new_query, embedding_model=embedding_model, document_embeddings=document_embeddings, top_k=1, verbose=True):
    # 쿼리 임베딩 계산
    new_query_embedding = embedding_model.encode([new_query])

    
    # 코사인 유사도 계산
    cos_sim = cosine_similarity(new_query_embedding, document_embeddings)
    
    # 코사인 유사도 값이 가장 큰 질문의 인덱스 찾기
    most_similar_idx = np.argmax(cos_sim)
    similarity = np.round(cos_sim[0][most_similar_idx], 2)
    
    # 가장 비슷한 질문과 답변 가져오기
    similar_query = query_texts[most_similar_idx]
    similar_answer = qna_df[qna_df.index == most_similar_idx]['답변'].to_string(index=False)
    
    if verbose == True:
        print("가장 비슷한 질문 : ", similar_query)
        print("가장 비슷한 질문의 유사도 : ", similarity)
        print("가장 비슷한 질문의 답: ", similar_answer)

    # 결과 반환
    return similar_query, similarity, similar_answer

# 함수 호출 예시
new_query = '발권출납 옛날 돈과 현용화폐와의 교환'

query_answer(new_query)

가장 비슷한 질문 :  발권출납 옛날 돈과 현용화폐와의 교환
가장 비슷한 질문의 유사도 :  1.0
가장 비슷한 질문의 답:  한국은행은 1962년 이후에 발행된 원 표시 화폐에 대해서만 액면 금액으로 교환해 드...


('발권출납 옛날 돈과 현용화폐와의 교환',
 1.0,
 '한국은행은 1962년 이후에 발행된 원 표시 화폐에 대해서만 액면 금액으로 교환해 드...')

In [80]:
new_query = '전세계 외환시장의 거래규모'
query_answer(new_query, verbose=False)[2]

'세계 외환거래규모와 관련하여 보통 이용하는 통계는 국제결제은행(BIS)의 「세계 외환...'